# Import 

In [52]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from xgboost  import XGBClassifier
import xgboost as xgb
from scipy.stats import randint
from sklearn.utils.fixes import loguniform



# Load data

In [17]:
train = pd.read_parquet('data/final_train.parquet')
test = pd.read_parquet('data/final_test.parquet')

# Encoder

- 삭제 : id, base_date,day_of_week,base_hour , end_latitude, end_longitude, start_latitude, start_longitude, start_node_name, end_node_name, road_name
  

- One-Hot Encoder  : rain_snow, start_turn_restricted, end_turn_restricted, day_of_week ,week,time 

In [18]:
del_col = ['id','base_date','day_of_week','end_latitude', 'end_longitude', 'start_latitude', 'start_longitude', 'start_node_name', 'end_node_name', 'road_name','day','month']
cat_col= ['rain_snow', 'start_turn_restricted', 'end_turn_restricted','vacation']

In [19]:
train_drop = train.drop(columns = del_col)
test_drop = test.drop(columns = del_col)

In [20]:
transformer = make_column_transformer((OneHotEncoder(), cat_col))

train_transformed = transformer.fit_transform(train_drop)
test_transformed = transformer.transform(test_drop)

train_tra = pd.DataFrame(train_transformed, columns=transformer.get_feature_names_out())
test_tra = pd.DataFrame(test_transformed, columns=transformer.get_feature_names_out())

In [21]:
train_drop = train_drop.drop(columns=cat_col)
test_drop = test_drop.drop(columns=cat_col)

In [22]:
train_data = pd.concat([train_drop, train_tra], axis=1)
test_data = pd.concat([test_drop, test_tra], axis=1)

In [23]:
train_data.head()

,base_hour,lane_count,road_rating,multi_linked,connect_code,maximum_speed_limit,weight_restricted,road_type,target,week,...,center_start,center_end,onehotencoder__rain_snow_눈비,onehotencoder__rain_snow_없음,onehotencoder__start_turn_restricted_없음,onehotencoder__start_turn_restricted_있음,onehotencoder__end_turn_restricted_없음,onehotencoder__end_turn_restricted_있음,onehotencoder__vacation_방학,onehotencoder__vacation_학기
0,17,1,106,0,0,60.0,32400.0,3,52.0,0.0,...,0,0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,21,2,103,0,0,60.0,0.0,0,30.0,0.0,...,1,1,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,7,2,103,0,0,80.0,0.0,0,61.0,2.0,...,0,0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,13,2,107,0,0,50.0,0.0,0,20.0,1.0,...,2,2,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
4,8,2,103,0,0,80.0,0.0,0,38.0,0.0,...,0,0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0


# Model

In [24]:
y_train = train_data['target']
X_train = train_data.drop(columns=['target'])

In [25]:
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,
                                                    test_size=0.2, random_state=42)

In [26]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(3760973, 22) (3760973,)
(940244, 22) (940244,)


## XGboost

In [53]:
model_xgb=xgb.XGBRegressor(random_state=42, n_jobs=-1)

In [54]:
param_grid = {'nthread':[4], 
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], 
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]
}

In [55]:
cv_xgb=RandomizedSearchCV(model_xgb,param_grid, cv=3)

In [56]:
cv_xgb.fit(X_train,y_train)

/Users/yeseulseo/miniforge3/envs/jeju/lib/python3.8/site-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[18:44:31] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[18:44:31] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "silent" } are not used.



KeyboardInterrupt: 

In [49]:

model_xgb1= xgb.XGBRegressor(n_estimators=10000, 
                              max_depth=5, 
                              subsample=0.3,
                              colsample_bytree=0.5,
                              reg_lambda=15,
                              random_state=42)

   

In [51]:

model_xgb1.fit(X_train,y_train)
               

KeyboardInterrupt: 

In [ ]:
pred_xgb1= model_xgb1.predict(X_val)

In [ ]:
mean_absolute_error(y_val, pred_xgb1)

4.663731015551993